# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
from pandas.tseries.offsets import *
import holidays
import pandas_datareader.data as web

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('IBOV.csv')
import yfinance as yf
yf.pdr_override()

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'BBAS3.SA'
#.SA para as BR
ticker = yf.Ticker(f'{symbol}')
stockinfo = ticker.info
stockinfo

data = web.get_data_yahoo(f'{symbol}')
data = data[['Close']]
    
#print(data.Close.iloc[len(data)-1])


[*********************100%***********************]  1 of 1 completed



## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data.Close.iloc[len(data)-1]

28.979999542236328

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 50))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

ALPA4,ABEV3,AMER3,ASAI3,AZUL4,B3SA3,BIDI4,BIDI11,BPAN4,BBSE3,BRML3,BBDC3,BBDC4,BRAP4,BBAS3,BRKM5,BRFS3,BPAC11,CRFB3,CCRO3,CMIG4,CIEL3,COGN3,CPLE6,CSAN3,CPFE3,CVCB3,CYRE3,DXCO3,ECOR3,ELET3,ELET6,EMBR3,ENBR3,ENGI11,ENEV3,EGIE3,EQTL3,EZTC3,FLRY3,GGBR4,GOAU4,GOLL4,NTCO3,SOMA3,HAPV3,HYPE3,IGTA3,GNDI3,IRBR3
ITSA4,ITUB4,JBSS3,JHSF3,KLBN11,RENT3,LCAM3,LWSA3,LAME4,LREN3,MGLU3,MRFG3,CASH3,BEEF3,MRVE3,MULT3,PCAR3,PETR3,PETR4,BRDT3,PRIO3,PETZ3,QUAL3,RADL3,RDOR3,RAIL3,SBSP3,SANB11,CSNA3,SULA11,SUZB3,TAEE11,VIVT3,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VIIA3,WEGE3,YDUQ3


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
#final_dataframe = pd.DataFrame(columns = my_columns)
#for symbol in stocks['Ticker']:
#    print(symbol)
#    data = web.get_data_yahoo(f'{symbol}')
#    pricetoday = data.Close.iloc[len(data)-1]
#    if len(data) >= 300:
#        priceoneyear = data.Close.iloc[len(data)-300]
#    else :
#        priceoneyear = data.Close.iloc[1]
#    oneyeareturn = priceoneyear - pricetoday
#    final_dataframe = final_dataframe.append(
#    pd.Series(
#         [
#         symbol,
#         pricetoday,
#         oneyeareturn,
#         'N/A'
#         ],
#         index = my_columns),
#    ignore_index = True,    
#    )
#    
#final_dataframe

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
#final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
#final_dataframe = final_dataframe[:50]
#final_dataframe.reset_index(inplace = True)
#final_dataframe

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
portfolio_size = 1000000
# portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [9]:
#position_size = val/len(final_dataframe.index)
#for i in range(0, len(final_dataframe.index)):
#    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Price'])
#
#final_dataframe

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'Two-Month Price Return',
    'Two-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol in stocks['Ticker']:
    data = web.get_data_yahoo(f'{symbol}.SA')
    pricetoday = data.Close.iloc[len(data)-1]
    print(symbol)
    if len(data) >= 300:
        priceoneyear = data.Close.iloc[len(data)-300]
    else :
        priceoneyear = data.Close.iloc[1]
    if len(data) >= 22*6:
        pricesixmonth = data.Close.iloc[len(data)-22*6]
    else :
        pricesixmonth = data.Close.iloc[1]
    if len(data) >= 22*3:
        pricethreemonth = data.Close.iloc[len(data)-22*3]
    else :
        pricethreemonth = data.Close.iloc[1]
    if len(data) >= 22*2:
        pricetwomonth = data.Close.iloc[len(data)-22*2]
    else :
        pricetwomonth = data.Close.iloc[1]
    twomonthreturn = pricetoday - pricetwomonth
    threemonthreturn = pricetoday - pricethreemonth
    sixmonthreturn = pricetoday - pricesixmonth
    oneyeareturn = pricetoday - priceoneyear
    hqm_dataframe = hqm_dataframe.append(
         pd.Series(
         [
             symbol,
             pricetoday,
             'N/A',
             oneyeareturn,
             'N/A',
             sixmonthreturn,
             'N/A',
             threemonthreturn,
             'N/A',
             twomonthreturn,
             'N/A',
             'N/A'
         ],
         index = hqm_columns),
     ignore_index = True
     )

hqm_dataframe        

[*********************100%***********************]  1 of 1 completed
ALPA4
[*********************100%***********************]  1 of 1 completed
ABEV3
[*********************100%***********************]  1 of 1 completed
AMER3
[*********************100%***********************]  1 of 1 completed
ASAI3
[*********************100%***********************]  1 of 1 completed
AZUL4
[*********************100%***********************]  1 of 1 completed
B3SA3
[*********************100%***********************]  1 of 1 completed
BIDI4
[*********************100%***********************]  1 of 1 completed
BIDI11
[*********************100%***********************]  1 of 1 completed
BPAN4
[*********************100%***********************]  1 of 1 completed
BBSE3
[*********************100%***********************]  1 of 1 completed
BRML3
[*********************100%***********************]  1 of 1 completed
BBDC3
[*********************100%***********************]  1 of 1 completed
BBDC4
[*********************10

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,ALPA4,40.509998,N/A,6.509998,N/A,2.000000,N/A,-11.060001,N/A,-17.880001,N/A,N/A
1,ABEV3,17.090000,N/A,3.610001,N/A,1.240000,N/A,-0.240000,N/A,0.049999,N/A,N/A
2,AMER3,29.870001,N/A,-90.390001,N/A,-34.930002,N/A,-23.129999,N/A,-12.190001,N/A,N/A
3,ASAI3,15.800000,N/A,1.300000,N/A,-0.432000,N/A,-1.815999,N/A,-1.599999,N/A,N/A
4,AZUL4,25.680000,N/A,2.800001,N/A,-13.230000,N/A,-15.000000,N/A,-13.489998,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
86,USIM5,13.660000,N/A,5.020000,N/A,-8.490000,N/A,-6.530001,N/A,-4.840000,N/A,N/A
87,VALE3,72.250000,N/A,10.750000,N/A,-35.739998,N/A,-45.050003,N/A,-28.440002,N/A,N/A
88,VIIA3,6.370000,N/A,-11.790000,N/A,-6.750000,N/A,-6.900001,N/A,-4.500000,N/A,N/A
89,WEGE3,37.160000,N/A,3.160000,N/A,-0.445000,N/A,-0.040001,N/A,0.259998,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'Two-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])        
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,ALPA4,40.509998,N/A,6.509998,85.7143,2.000000,92.3077,-11.060001,12.0879,-17.880001,3.2967,N/A
1,ABEV3,17.090000,N/A,3.610001,75.8242,1.240000,89.011,-0.240000,82.4176,0.049999,89.011,N/A
2,AMER3,29.870001,N/A,-90.390001,1.0989,-34.930002,2.1978,-23.129999,3.2967,-12.190001,10.989,N/A
3,ASAI3,15.800000,N/A,1.300000,62.6374,-0.432000,83.5165,-1.815999,71.4286,-1.599999,67.033,N/A
4,AZUL4,25.680000,N/A,2.800001,70.3297,-13.230000,12.0879,-15.000000,8.79121,-13.489998,7.69231,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
86,USIM5,13.660000,N/A,5.020000,81.3187,-8.490000,20.8791,-6.530001,36.2637,-4.840000,38.4615,N/A
87,VALE3,72.250000,N/A,10.750000,92.3077,-35.739998,1.0989,-45.050003,1.0989,-28.440002,2.1978,N/A
88,VIIA3,6.370000,N/A,-11.790000,6.59341,-6.750000,31.8681,-6.900001,29.6703,-4.500000,40.6593,N/A
89,WEGE3,37.160000,N/A,3.160000,71.4286,-0.445000,82.4176,-0.040001,85.7143,0.259998,91.2088,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_precentiles = []
    for time_period in time_periods:
        momentum_precentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_precentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,ALPA4,40.509998,N/A,6.509998,85.7143,2.000000,92.3077,-11.060001,12.0879,-17.880001,3.2967,48.3516
1,ABEV3,17.090000,N/A,3.610001,75.8242,1.240000,89.011,-0.240000,82.4176,0.049999,89.011,84.0659
2,AMER3,29.870001,N/A,-90.390001,1.0989,-34.930002,2.1978,-23.129999,3.2967,-12.190001,10.989,4.3956
3,ASAI3,15.800000,N/A,1.300000,62.6374,-0.432000,83.5165,-1.815999,71.4286,-1.599999,67.033,71.1538
4,AZUL4,25.680000,N/A,2.800001,70.3297,-13.230000,12.0879,-15.000000,8.79121,-13.489998,7.69231,24.7253
...,...,...,...,...,...,...,...,...,...,...,...,...
86,USIM5,13.660000,N/A,5.020000,81.3187,-8.490000,20.8791,-6.530001,36.2637,-4.840000,38.4615,44.2308
87,VALE3,72.250000,N/A,10.750000,92.3077,-35.739998,1.0989,-45.050003,1.0989,-28.440002,2.1978,24.1758
88,VIIA3,6.370000,N/A,-11.790000,6.59341,-6.750000,31.8681,-6.900001,29.6703,-4.500000,40.6593,27.1978
89,WEGE3,37.160000,N/A,3.160000,71.4286,-0.445000,82.4176,-0.040001,85.7143,0.259998,91.2088,82.6923


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,JBSS3,39.650002,N/A,17.280001,96.7033,4.650002,95.6044,7.730001,100,8.600002,100,98.0769
1,MRFG3,26.610001,N/A,11.890000,93.4066,6.120001,98.9011,6.540001,98.9011,5.610001,98.9011,97.5275
2,PRIO3,23.719999,N/A,14.839999,95.6044,5.349998,96.7033,5.719999,97.8022,4.320000,97.8022,96.978
3,EMBR3,22.209999,N/A,14.409999,94.5055,6.699999,100,2.839998,95.6044,-1.470001,71.4286,90.3846
4,PETR3,28.700001,N/A,4.650002,79.1209,5.430000,97.8022,0.360001,90.1099,-0.379999,84.6154,87.9121
...,...,...,...,...,...,...,...,...,...,...,...,...
86,NTCO3,40.130001,N/A,-6.489998,18.6813,-7.869999,24.1758,-14.869999,9.89011,-12.669998,9.89011,15.6593
87,MGLU3,11.060000,N/A,-8.954999,10.989,-10.669999,15.3846,-10.470000,13.1868,-7.970000,16.4835,14.011
88,RENT3,45.529999,N/A,-4.920002,26.3736,-16.170002,8.79121,-20.610001,6.59341,-12.770000,8.79121,12.6374
89,EZTC3,18.520000,N/A,-18.980000,4.3956,-15.709999,9.89011,-9.879999,15.3846,-9.439999,14.2857,10.989


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portifolio = 1000000
#portifolio_input()

In [15]:
position_size = float(portifolio)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
     hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,JBSS3,39.650002,277,17.280001,96.7033,4.650002,95.6044,7.730001,100,8.600002,100,98.0769
1,MRFG3,26.610001,412,11.890000,93.4066,6.120001,98.9011,6.540001,98.9011,5.610001,98.9011,97.5275
2,PRIO3,23.719999,463,14.839999,95.6044,5.349998,96.7033,5.719999,97.8022,4.320000,97.8022,96.978
3,EMBR3,22.209999,494,14.409999,94.5055,6.699999,100,2.839998,95.6044,-1.470001,71.4286,90.3846
4,PETR3,28.700001,382,4.650002,79.1209,5.430000,97.8022,0.360001,90.1099,-0.379999,84.6154,87.9121
...,...,...,...,...,...,...,...,...,...,...,...,...
86,NTCO3,40.130001,273,-6.489998,18.6813,-7.869999,24.1758,-14.869999,9.89011,-12.669998,9.89011,15.6593
87,MGLU3,11.060000,993,-8.954999,10.989,-10.669999,15.3846,-10.470000,13.1868,-7.970000,16.4835,14.011
88,RENT3,45.529999,241,-4.920002,26.3736,-16.170002,8.79121,-20.610001,6.59341,-12.770000,8.79121,12.6374
89,EZTC3,18.520000,593,-18.980000,4.3956,-15.709999,9.89011,-9.879999,15.3846,-9.439999,14.2857,10.989


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momemtum_strategy_IBOV.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name = "Momemtum Strategy Br IBOV", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats ={
       'A': ['Ticker', string_template],
       'B': ['Price', dollar_template],
       'C': ['Number of Shares to Buy', integer_template],
       'D': ['One-Year Price Return', percent_template],
       'E': ['One-Year Return Percentile', percent_template],
       'F': ['Six-Month Price Return', percent_template],
       'G': ['Six-Month Return Percentile', percent_template],
       'H': ['Three-Month Price Return', percent_template],
       'I': ['Three-Month Return Percentile', percent_template],
       'J': ['One-Month Price Return', percent_template],
       'K': ['One-Month Return Percentile', percent_template],
       'L': ['HQM Score', integer_template]        
}

for column in column_formats.keys():
    writer.sheets['Momemtum Strategy Br IBOV'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momemtum Strategy Br IBOV'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()